<h1 style="text-align:center;"> Graphene Model - Again</h1>
<div style="height:100px;"></div>
<h3 style="text-align:center;"> Anoop </h3>
<h4 style="text-align:center;">
    <span style="color:#777">(github.com/strivetobelazy/graphene-model)</span>
</h4>

In [ ]:
import numpy as np
import scipy.linalg as spl

%matplotlib inline
import matplotlib
import matplotlib.pylab as plt

from sympy import Symbol,sin,cos,nsolve
from scipy.optimize import minimize
from scipy.optimize import root

from ipywidgets import interact, fixed
import ipywidgets as widgets

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
from IPython.core.display import display, HTML

# The polling here is to ensure that plotly.js has already been loaded before
# setting display alignment in order to avoid a race condition.
display(HTML(
    '<script>'
        'var waitForPlotly = setInterval( function() {'
            'if( typeof(window.Plotly) !== "undefined" ){'
                'MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });'
                'MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);'
                'clearInterval(waitForPlotly);'
            '}}, 250 );'
    '</script>'
))
%config InlineBackend.figure_format = 'retina'

<!-- Equation and Algorithm-->
<!--<img src="sp2.png" style="display: block;margin:auto;width:40%;float:left;margin-top:60px">-->
<img src="color_sublattice.png" style="display: block;margin:auto;width:45%;">

$$\vec{a1}=a\left(-\frac{1}{2}\vec{e_x}+\frac{\sqrt{3}}{2}\vec{e_y}\right), \vec{a2}=a\left(\frac{1}{2}\vec{e_x}+\frac{\sqrt{3}}{2}\vec{e_y}\right)$$

$$\vec{k1}=2\pi a^{-1}\left(-\vec{e_x}+\frac{1}{\sqrt{3}}\vec{e_y}\right), \vec{k2}=2\pi a^{-1}\left(\vec{e_x}+\frac{1}{\sqrt{3}}\vec{e_y}\right)$$

$$\hat{H}=-\frac{1}{2}\nabla^2+\sum_{\vec{R}}U\left(\vec{r}-\vec{R}\right)\\
\hat{H}=-\frac{1}{2}\nabla^2+\sum_{\vec{R}}\sum_{\vec{r_n}}U_0\left(\vec{r}-\vec{R}-\vec{r_n}\right)$$


$$\psi(\vec{r})=\sum_n\sum_{\vec{R}}e^{i\vec{k}\vec{R}}b_n\phi(\vec{r}-\vec{R}-\vec{r_n})$$

$$Eb_m=t\sum_{(n,\vec{R})\in\sigma_m}b_n e^{i\vec{k}\vec{R}}$$

### Graphene Band Structure

In [ ]:
a=1;t=-2.6; a0=[0,0]
a1=[-1.*a/2.,a*np.sqrt(3.)/2.]
a2=[1.*a/2.,a*np.sqrt(3.)/2.]
b1 = [1.*a/2.,-1*a*np.sqrt(3.)/2.] #-a1
b2 = [-1.*a/2.,-1*a*np.sqrt(3.)/2.] #-a2

In [ ]:
sigma=[]
for i in range(2):
    if (i==0): sigma.append([[1,a1],[1,a2],[1,a0]])
    if (i==1): sigma.append([[0,b1],[0,b2],[0,a0]])

def Hamiltonian(kx,ky):
    H = np.zeros((2, 2), dtype=np.complex)
    kvec=[kx,ky]
    for m in range(2):
        for n in range(2):
            if (m!=n):
                for i in range(3):
                    Rvec=sigma[m][i][1]
                    H[m,n] +=  np.complex(t*np.exp(1j*np.vdot(kvec,Rvec)))
    return H

In [ ]:
def plot_graphene_3D_N(n,xlim,ylim,resol):
    x = np.arange(xlim,ylim ,resol)
    y = np.arange(xlim,ylim ,resol)

    kx, ky  =  np.mgrid[xlim:ylim:resol, xlim:ylim:resol]
    # ZN = np.zeros_like(kx)

    ZN = np.zeros((n,kx.shape[0],kx.shape[0]))
    e=np.zeros(n)

    for i in range(kx.shape[0]):
        for j in range(kx.shape[1]):
            for m in range(n):
                H = Hamiltonian(kx[i,j],ky[i,j])
                evals, evecs = spl.eigh(H)
                ZN[m,i,j] = evals[m]

    data=[]
    for i in range(n):
        trace=go.Surface(
                x=x,
                y=y,
                z=ZN[i],
                showscale=True if i == 1 else False
            )
        data.append(trace)

    layout = go.Layout(
        title='Graphine-Numerical',
            margin=dict(
            l=65,
            r=50,
            b=65,
            t=90
        ),
    )
    fig2 = go.Figure(data=data, layout=layout)
    py.iplot(fig2, filename='3d-surface-graphene-numerical',show_link=False)
    return x,y,ZN

In [ ]:
n=2;lmin = -4*np.pi/3.;lmax = 4*np.pi/3.
x,y,ZN=plot_graphene_3D_N(n,lmin,lmax,0.1)

<img src="zGNR.png" style="display: block;margin:auto;width:30%;">

### Graphene ZigZag Ribbon(zGNR)

In [ ]:
t=2.6; a0=0.; a1=1.

def sigma_z(m,kvec):
    ct=np.complex(t)
    nt=t*(1+np.exp(-1j*kvec*a1))
    pt=t*(1+np.exp(1j*kvec*a1))
    if (m%4==0) : pos1=m+1;val1=ct; pos2=m-1;val2=nt
    if (m%4==1) : pos1=m-1;val1=ct; pos2=m+1;val2=nt
    if (m%4==2) : pos1=m+1;val1=ct; pos2=m-1;val2=pt
    if (m%4==3) : pos1=m-1;val1=ct; pos2=m+1;val2=pt
    return pos1,val1,pos2,val2

def hamiltonian_z(N, kvec, edge=False, edgevalue=0.7*t):
    Eb = edgevalue
    H = np.zeros((2*N, 2*N), dtype=np.complex)
    for m in range(2*N):
        pos1,val1,pos2,val2 = sigma_z(m+1,kvec)
        if (pos1>0) and (pos1<2*N+1): H[m,pos1-1]=val1
        if (pos2>0) and (pos2<2*N+1): H[m,pos2-1]=val2
        if edge==True : H[0,0]=Eb;H[2*N-1,2*N-1]=Eb
    return H

In [ ]:
def band_struct_z(N,xlim, ylim ,resol,edge,edgevalue):
    x = np.arange(xlim, ylim ,resol)
    zGNR = np.zeros((len(x),2*N))
    for i in range(len(x)):
        H = hamiltonian_z(N,x[i],edge,edgevalue)
        evals, evecs = spl.eigh(H)
        zGNR[i,:] = evals
    return x, zGNR


def plot_graphene_nanoribbons_Z(N, xlim, ylim,resol, edge, edgevalue):
    x,zGNR=band_struct_z(N,xlim, ylim ,resol, edge, edgevalue)
    # #plotting
    data=[]
    for i in range(N):
        trace1=go.Scatter(
                x=x,
                y=zGNR[:,i],
                name='-E'
            )
        data.append(trace1)
        trace2=go.Scatter(
                x=x,
                y=-zGNR[:,i],
                name='+E'
            )
        data.append(trace2)
    if edge==True:
        title = 'Graphine-nanoribbons(zGNR) with edge( Eb = '+str(round(edgevalue/t,1))+'t, '+str(N)+' atoms )'
    else:
        title = 'Graphine-nanoribbons(zGNR) without edge( '+str(N)+' atoms )'
    layout = go.Layout(
        title=title,
        xaxis=dict(title = '<b>k</b>'),        
        yaxis=dict(title = '<b>E(k)</b>'),
        shapes= [
        {
            'type': 'rect',
            'x0': np.pi,
            'y0': -8.,
            'x1': np.pi,
            'y1': 8,
            'line': {
                'color': 'rgb(66,165,245)',
                'width': 2,
                'dash': 'dot',
            },
        },
        {
            'type': 'rect',
            'x0': -1.*np.pi,
            'y0': -8.,
            'x1': -1.*np.pi,
            'y1': 8,
            'line': {
                'color': 'rgb(66,165,245)',
                'width': 2,
                'dash': 'dot',
            },
        },
        ],
        showlegend=False,
            margin=dict(
            l=65,
            r=50,
            b=65,
            t=90
        ),
    )
    fig2 = go.Figure(data=data, layout=layout)
    py.iplot(fig2, filename='zGNR',show_link=False)

slider_zGNR=widgets.SelectionSlider(
    options={'0': 0, '1': 1, '2': 2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, 
             '20':20, '30':30, '50':50, '80':80, '100':100},
    value=6,
    description='N',
    disabled=False,
    orientation='horizontal',
    readout=True,
)

xlim = -1.2*np.pi; ylim = 1.2*np.pi; resol=0.1
def iplotz(i, edge, edgevalue):
    plot_graphene_nanoribbons_Z(i, xlim, ylim, resol, edge, edgevalue)
    

In [ ]:
interact(iplotz,i=slider_zGNR, edge=fixed(False), edgevalue=fixed(0.7*t));

In [ ]:
interact(iplotz,i=slider_zGNR, edge=fixed(True), edgevalue=fixed(0.7*t));

In [ ]:
Delta = np.zeros(100)
for Nat in range(1,101):
    Delta[Nat-1]=4*np.pi/Nat

data=[]
trace1=go.Scatter(
        y=Delta[:],
        name='delta',
        mode='lines+markers'
    )
data.append(trace1)

layout = go.Layout(
title='Band gap of the zigzag ribbon at k = 2π/3',
xaxis=dict(title = '<b>N</b>'),        
yaxis=dict(title = '<b>delta(N)</b>'),
showlegend=False,
    margin=dict(
    l=65,
    r=50,
    b=65,
    t=90
),
)
fig2 = go.Figure(data=data, layout=layout)

In [ ]:
py.iplot(fig2, filename='aGNR',show_link=False)

### Graphene Armchair Ribbon(aGNR)

<img src="aGNR.png" style="display: block;margin:auto;width:40%;">

In [ ]:
# aGNR
t=2.6; a0=0.; a1=1.

def sigma_a(m,kvec):
    ct=np.complex(t)
    nt=t*np.exp(-1j*kvec*a1)
    pt=t*np.exp(1j*kvec*a1)
    if (m%4==0) : pos1=m-3;val1=ct; pos2=m+1;val2=ct; pos3=m-1;val3=nt
    if (m%4==1) : pos1=m+3;val1=ct; pos2=m+1;val2=ct; pos3=m-1;val3=ct
    if (m%4==2) : pos1=m-3;val1=ct; pos2=m-1;val2=ct; pos3=m+1;val3=ct
    if (m%4==3) : pos1=m+3;val1=ct; pos2=m-1;val2=ct; pos3=m+1;val3=pt
    return pos1, val1, pos2, val2, pos3, val3

def hamiltonian_a(N, kvec, edge=False, edgevalue=0.7*t):
    Eb = edgevalue
    H = np.zeros((2*N, 2*N), dtype=np.complex)
    for m in range(2*N):
        pos1,val1,pos2,val2,pos3,val3 = sigma_a(m+1,kvec)
        if (pos1>=1) and (pos1<=2*N): H[m,pos1-1]=val1
        if (pos2>=1) and (pos2<=2*N): H[m,pos2-1]=val2        
        if (pos3>=1) and (pos3<=2*N): H[m,pos3-1]=val3
        if edge==True : H[0,0]=Eb;H[2*N-1,2*N-1]=Eb; #H[1,1]=Eb;H[2*N-2,2*N-2]=Eb
    return H

In [ ]:
def band_struct_a(N,xlim, ylim ,resol,edge,edgevalue):
    x = np.arange(xlim, ylim ,resol)
    aGNR = np.zeros((len(x),2*N))
    for i in range(len(x)):
        H = hamiltonian_a(N,x[i],edge,edgevalue)
        evals, evecs = spl.eigh(H)
        aGNR[i,:] = evals
    return x, aGNR

In [ ]:
def plot_graphene_nanoribbons_A(N, xlim, ylim,resol, edge, edgevalue):
    x,aGNR=band_struct_a(N, xlim, ylim ,resol, edge, edgevalue)
    # #plotting
    data=[]
    for i in range(N):
        trace1=go.Scatter(
                x=x,
                y=aGNR[:,i],
                name='-E'
            )
        data.append(trace1)
        trace2=go.Scatter(
                x=x,
                y=-aGNR[:,i],
                name='+E'
            )
        data.append(trace2)
        
    if edge==True:
        title = 'Graphine-nanoribbons(aGNR) with edge( Eb = '+str(round(edgevalue/t,1))+'t, '+str(N)+' atoms )'
    else:
        title = 'Graphine-nanoribbons(aGNR) without edge( '+str(N)+' atoms )'
        
    layout = go.Layout(
        title=title,
        xaxis=dict(title = '<b>k</b>'),        
        yaxis=dict(title = '<b>E(k)</b>'),
        showlegend=False,
            margin=dict(
            l=65,
            r=50,
            b=65,
            t=90
        ),
    )
    fig2 = go.Figure(data=data, layout=layout)
    py.iplot(fig2, filename='aGNR',show_link=False)

slider_aGNR=widgets.SelectionSlider(
    options={'0': 0, '1': 1, '2': 2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, '15':15,
             '20':20, '30':30, '50':50, '80':80, '100':100},
    value=6,
    description='np',
    disabled=False,
    orientation='horizontal',
    readout=True,
)

xlim = -1.*np.pi; ylim = np.pi; resol=0.1
def iplotz(i, edge, edgevalue):
    plot_graphene_nanoribbons_A(i, xlim, ylim, resol, edge, edgevalue)
    

In [ ]:
interact(iplotz,i=slider_aGNR, edge=fixed(False), edgevalue=fixed(0.7*t));

In [ ]:
interact(iplotz,i=slider_aGNR, edge=fixed(True), edgevalue=fixed(0.7*t));

## Density of states

$$D(E)= \sum_m\delta(E-E_m)$$

In [ ]:
E=ZN.flatten()
dos_n,bin_edges_n = np.histogram(E,80,normed=True)
# dos_p,bin_edges_p = np.histogram(-E,80,normed=True)


# trace0 = go.Scatter(
#     x = bin_edges_p,
#     y = dos_p,
#     mode = 'lines',
# )
trace1 = go.Scatter(
    x = bin_edges_n,
    y = dos_n,
    mode = 'lines',
)

data_dos = [trace1]
layout_dos = dict(
    title = '<b>DOS-Graphene</b>',
    xaxis = dict(title = '<b>E</b>'),
    yaxis = dict(title = '<b>D(E)</b>'),
    showlegend=False,
    )

fig_dos = dict(data=data_dos, layout=layout_dos)

In [ ]:
py.iplot(fig_dos, filename='dos-graphene',show_link=False)

In [ ]:
def plot_dos_zGNR(N,xlim, ylim ,resol, edge, edgevalue):
    x,zGNR=band_struct_z(N,xlim, ylim ,resol, edge, edgevalue)
    zE=zGNR.flatten()
    dos_n,bin_edges_n = np.histogram(zE,80,normed=True)
    # dos_p,bin_edges_p = np.histogram(-zE,80,normed=True)


    trace0 = go.Scatter(
        x = bin_edges_n,
        y = dos_n,
        mode = 'lines',
    )
    # trace1 = go.Scatter(
    #     x = bin_edges_p,
    #     y = dos_p,
    #     mode = 'lines',
    # )

    data_dos = [trace0]
    if edge==True: 
        title = '<b>DOS-zGNR with Edge effect(Eb = '+str(round(edgevalue/t,1))+'t )</b>'
    else:
        title = '<b>DOS-zGNR without Edge effect</b>'
    layout_dos = dict(
        title = title,
        xaxis = dict(title = '<b>E</b>'),
        yaxis = dict(title = '<b>D(E)</b>'),
        showlegend=False,
        )

    fig_dos = dict(data=data_dos, layout=layout_dos)
    py.iplot(fig_dos, filename='dos-graphene',show_link=False)

slider_zGNR_dos=widgets.SelectionSlider(
    options={'0': 0, '0.1': 1, '0.2': 2, '0.3':3, '0.4':4, '0.5':5, '0.6':6, '0.7':7, '0.8':8, '0.9':9, 
             '1.0':10, '1.1':11, '1.2':12, '1.3':13, '1.4':14},
    value=0,
    description='t',
    disabled=False,
    orientation='horizontal',
    readout=True,
)

def iplotdos(i,edge):
        xlim = -1*np.pi; ylim = np.pi; resol=0.1; edge=edge; edgevalue=0.1*i*t;N=100
        plot_dos_zGNR(N,xlim,ylim,resol,edge,edgevalue)

In [ ]:
interact(iplotdos, i=slider_zGNR_dos, edge=fixed(True));

In [ ]:
xlim = -1*np.pi; ylim = np.pi; resol=0.1; edge=False; edgevalue=1.0*t;N=100
x,aGNR=band_struct_a(N,xlim, ylim ,resol, edge, edgevalue)

aE=aGNR.flatten()
dos_a,bin_edges_a = np.histogram(aE,80,normed=True)

trace0 = go.Scatter(
    x = bin_edges_a,
    y = dos_a,
    mode = 'lines',
)

data_dos = [trace0]
if edge==True: 
    title = '<b>DOS-aGNR with Edge effect</b>'
else:
    title = '<b>DOS-aGNR without Edge effect</b>'
layout_dos = dict(
    title = title,
    xaxis = dict(title = '<b>E</b>'),
    yaxis = dict(title = '<b>D(E)</b>'),
    showlegend=False,
    )

fig_dos = dict(data=data_dos, layout=layout_dos)

In [ ]:
py.iplot(fig_dos, filename='dos-graphene',show_link=False)